In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
plt.rc('text', usetex=True)
plt.rc('font', family='Serif')

import matplotlib as mpl 
mpl.rcParams['figure.figsize'] = [10, 7]
mpl.rcParams['font.size'] = 27

mpl.rcParams['savefig.dpi'] = 150 
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['xtick.major.size'] = 10 
mpl.rcParams['xtick.minor.size'] = 5 

mpl.rcParams['ytick.major.size'] = 10 
mpl.rcParams['ytick.minor.size'] = 5 

mpl.rcParams['xtick.labelsize'] = 23
mpl.rcParams['ytick.labelsize'] = 23

In [ ]:
NGC3201_path = os.getcwd()
df = pd.read_csv("{}/NGC3201_DRCR.dat".format(NGC3201_path),sep='\s+',names=['ID', 'F275W', 'F336W', 'F438W', 'Vvega', 'Ivega'])

In [ ]:
df

In [ ]:
#determine MSTO and SGB
plt.scatter(df['Vvega'] - df['Ivega'],df['Vvega'],s=0.5,alpha=1,c='grey')
plt.xlim(0.6,1.0)
plt.ylim(16.5,20)
plt.scatter(0.712,18.1,s=0.7,c='r',label='MSTO')
plt.scatter(0.712+0.05,17.45,s=0.7,c='b',label='SGB')
plt.legend()
plt.gca().invert_yaxis()

In [ ]:
#plot the v vs i image within 2 mag from sgb
V_min = 17.45 - 2
V_max = 17.45 + 2
plt.scatter(df['Vvega'],df['Ivega'],s=0.5)
plt.xlim(V_min, V_max)
plt.ylim(13.5,19.5)
plt.savefig("{}/vi.png".format(NGC3201_path),dpi=500);

In [ ]:
#Selected ridgeline points
ridgeline = pd.read_csv("{}/NGC3201_ridgeline_points_DRCR.csv".format(NGC3201_path),names=['Vvega','Ivega'])

In [ ]:
df_first_cut = df[(df['Vvega'] < V_max) & (df['Vvega'] > V_min)]
xp = ridgeline['Vvega'].values
fp = ridgeline['Ivega'].values
plt.scatter(df_first_cut['Vvega'],df_first_cut['Ivega'],s=0.1)
plt.xlabel('Vvega')
plt.ylabel('Ivega')
plt.plot(xp,fp,c='r')
plt.ylim(13.5,19.5);

In [ ]:
Vvega = df_first_cut['Vvega'].values
Ivega = df_first_cut['Ivega'].values
d = []
for i in range(len(Vvega)):
    x = Vvega[i]
    ivega_needed = np.interp(x,xp,fp) #new linear interpolation
    #use slightly bigger cut to avoid deleting too much points. A finer cut will be applied once the fiducial cmd is generated
    if (((ivega_needed + .08) > Ivega[i]) & (Ivega[i] > (ivega_needed - .08))):
        d.append(df_first_cut.iloc[i][['Vvega','Ivega']].values.tolist())
df_second_cut = pd.DataFrame(d,columns=['v','i'])

In [ ]:
plt.scatter(df_second_cut['v'] - df_second_cut['i'],df_second_cut['v'],s=0.1)

In [ ]:
df_second_cut.to_csv("{}/NGC3201_fitstars_DRCR.dat".format(NGC3201_path),sep=',',index=False)